### We will colect text data using Twitter API.

In [1]:
# communicate with twitter api
!pip install tweepy

In [2]:
# read or write excell file
!pip install openpyxl     

In [3]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler  # communicate with twitter

In [4]:
consumer_key = "cEQU7HP8sIz5q598uA65g9ooa"
consumer_secret = "GLPgPqHoKyvQTNC5m0Fyd7JaSDE8ONPFKOIjB1dpBuWWuFhVo1"
access_token = "1321518103482695680-7WmynEdqVf8ozt0qYteptjpHejBs30"
access_token_secret = "qHTjoZdsvsAOYDAED7PahuMuNWyrZL0hquUFnSUK4HwMW"

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [6]:
query = "bjp election in bihar"

In [7]:
Tweets = api.search(query, count = 10, lang = 'en', exclude = 'retweets', tweet_mode = 'extended')
# tweet_mode = 'extended' - full text attribute
# tweet_mode = 'compat' - extract tweets having only 140 characters

for tweet in Tweets:
    print(tweet,"\n")
    print("........................................\n")

Status(_api=<tweepy.api.API object at 0x000002FA89A1FC48>, _json={'created_at': 'Mon Nov 02 12:27:26 +0000 2020', 'id': 1323240298785333248, 'id_str': '1323240298785333248', 'full_text': "#BiharElections2020 | Siwan has been BJP stronghold since 2005 and the party is aiming to win it for a fourth time this year\n\nTop contenders are Om Prakash Yadav from the BJP, RJD's Awadh Bihari Chaudhary and RLSP's Abdul Rizwan Ansari \n\n#ElectionsWithHT\n\nhttps://t.co/JODQtveGhJ https://t.co/qsriGPRlni", 'truncated': False, 'display_text_range': [0, 279], 'entities': {'hashtags': [{'text': 'BiharElections2020', 'indices': [0, 19]}, {'text': 'ElectionsWithHT', 'indices': [238, 254]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/JODQtveGhJ', 'expanded_url': 'https://www.hindustantimes.com/bihar-election/bihar-assembly-election-2020-bjp-aims-to-secure-fourth-straight-win-in-siwan/story-kuFY9YEwkgRD5FQg0ADqVN.html', 'display_url': 'hindustantimes.com/bihar-election…', 'indices

### Get Tweets and some Attributes

tweepy.Cursor() - allow us to get as many tweets as we want

In [8]:
df = pd.DataFrame(columns= ['Tweets', 'Users', 'User_status_count', 'user_followers',
                           'user_location', 'user_verified', 'fav_count', 'rt_count',
                           'tweet_date'])
print(df)
print(df.shape)

Empty DataFrame
Columns: [Tweets, Users, User_status_count, user_followers, user_location, user_verified, fav_count, rt_count, tweet_date]
Index: []
(0, 9)


In [9]:
def stream(data, file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_status_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'user_location'] = tweet.user.location
        df.loc[i, 'user_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [10]:
stream(data=['bjp election in bihar'],file_name='my_tweets')

In [11]:
df.head()

,Tweets,Users,User_status_count,user_followers,user_location,user_verified,fav_count,rt_count,tweet_date,User
0,#BiharElections2020 | Siwan has been BJP stron...,NaN,796313,7801126,India,True,0,0,2020-11-02 12:27:26,Hindustan Times
1,"RT @NarundarM: ""Baba Ka Dhaba"" gathered more c...",NaN,27828,493,"Reston, VA",False,0,73,2020-11-02 12:26:19,Rofl bhakt 🏹
2,"After PM Modi's speech in Chhapra, women suppo...",NaN,99816,1226,India,False,0,0,2020-11-02 12:23:35,हरीश भारतीय
3,RT @SurajKrBauddh: Without taking name (help) ...,NaN,1186,51869,"New Delhi, India",True,0,13,2020-11-02 12:23:26,Suraj Kumar Bauddh
4,"RT @NarundarM: ""Baba Ka Dhaba"" gathered more c...",NaN,12471,421,,False,0,73,2020-11-02 12:20:34,कैलाश प्रधान


In [12]:
!pip install textblob 

#### return sentiment(polarity, subjectivity)

#### Polarity score : [-1, +1]

#### Subjectivity : [0.0, 1.0]

#### where 0.0 is very objectivity and 1.0 is very subjective

In [13]:
from textblob import TextBlob

In [14]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ',tweet).split())

In [15]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [16]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [17]:
df.head()

,Tweets,Users,User_status_count,user_followers,user_location,user_verified,fav_count,rt_count,tweet_date,User,clean_tweet
0,#BiharElections2020 | Siwan has been BJP stron...,NaN,796313,7801126,India,True,0,0,2020-11-02 12:27:26,Hindustan Times,BiharElections2020 Siwan has been BJP strongho...
1,"RT @NarundarM: ""Baba Ka Dhaba"" gathered more c...",NaN,27828,493,"Reston, VA",False,0,73,2020-11-02 12:26:19,Rofl bhakt 🏹,RT Baba Ka Dhaba gathered more crowd than BJP ...
2,"After PM Modi's speech in Chhapra, women suppo...",NaN,99816,1226,India,False,0,0,2020-11-02 12:23:35,हरीश भारतीय,After PM Modi s speech in Chhapra women suppor...
3,RT @SurajKrBauddh: Without taking name (help) ...,NaN,1186,51869,"New Delhi, India",True,0,13,2020-11-02 12:23:26,Suraj Kumar Bauddh,RT Without taking name help of Pakistan BJP ca...
4,"RT @NarundarM: ""Baba Ka Dhaba"" gathered more c...",NaN,12471,421,,False,0,73,2020-11-02 12:20:34,कैलाश प्रधान,RT Baba Ka Dhaba gathered more crowd than BJP ...


In [18]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x))
df.head()

,Tweets,Users,User_status_count,user_followers,user_location,user_verified,fav_count,rt_count,tweet_date,User,clean_tweet,Sentiment
0,#BiharElections2020 | Siwan has been BJP stron...,NaN,796313,7801126,India,True,0,0,2020-11-02 12:27:26,Hindustan Times,BiharElections2020 Siwan has been BJP strongho...,Positive
1,"RT @NarundarM: ""Baba Ka Dhaba"" gathered more c...",NaN,27828,493,"Reston, VA",False,0,73,2020-11-02 12:26:19,Rofl bhakt 🏹,RT Baba Ka Dhaba gathered more crowd than BJP ...,Positive
2,"After PM Modi's speech in Chhapra, women suppo...",NaN,99816,1226,India,False,0,0,2020-11-02 12:23:35,हरीश भारतीय,After PM Modi s speech in Chhapra women suppor...,Neutral
3,RT @SurajKrBauddh: Without taking name (help) ...,NaN,1186,51869,"New Delhi, India",True,0,13,2020-11-02 12:23:26,Suraj Kumar Bauddh,RT Without taking name help of Pakistan BJP ca...,Positive
4,"RT @NarundarM: ""Baba Ka Dhaba"" gathered more c...",NaN,12471,421,,False,0,73,2020-11-02 12:20:34,कैलाश प्रधान,RT Baba Ka Dhaba gathered more crowd than BJP ...,Positive


In [19]:
n = 1
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 RT @NarundarM: "Baba Ka Dhaba" gathered more crowd than BJP leaders election rallies in Bihar.

Clean tweet:
 RT Baba Ka Dhaba gathered more crowd than BJP leaders election rallies in Bihar

Sentiment tweet:
 Positive


In [20]:
n = 25
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 RT @IMurugesh: After PM Modi's speech in Chhapra, women supporters celebrate on Bhojpuri songs
https://t.co/Qew5QKVQBW

via NaMo App https:…

Clean tweet:
 RT After PM Modi s speech in Chhapra women supporters celebrate on Bhojpuri songs via NaMo App https

Sentiment tweet:
 Neutral


In [21]:
n = 500
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 RT @TimesNow: Most anticipated election interview on TIMES NOW. 

Is Nitish the CM face and BJP the big boss?

Who is the vote puller: PM @…

Clean tweet:
 RT Most anticipated election interview on TIMES NOW Is Nitish the CM face and BJP the big boss Who is the vote puller PM

Sentiment tweet:
 Positive


In [22]:
n = 777
print("Original tweet:\n", df['Tweets'][n])
print()
print("Clean tweet:\n", df['clean_tweet'][n])
print()
print("Sentiment tweet:\n", df['Sentiment'][n])

Original tweet:
 RT @AhmedBilal_JK: In fiirst phase of Bihar election, people voted in favour of Mahagathbandhan.

JDU &amp; BJP abandoned labourers during lock…

Clean tweet:
 RT JK In fiirst phase of Bihar election people voted in favour of Mahagathbandhan JDU amp BJP abandoned labourers during lock

Sentiment tweet:
 Neutral


In [23]:
df[df.Sentiment == 'Positive'].shape[0]

237

In [24]:
df[df.Sentiment == 'Neutral'].shape[0]

698

In [25]:
df[df.Sentiment == 'Negative'].shape[0]

65